In [13]:
import mnist
import scipy.misc
import sys
import os
import torch
from torch import nn
import torchvision
from torchvision import datasets as torchsets
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [9]:
if sys.version_info[0] < 3:
    raise Excpetion('You must use Python 3 or higher.')

In [12]:
mnist_train = torchsets.MNIST(root='./data', train=True, download=True, transform=None)
mnist_test = torchsets.MNIST(root='./data', train=False, download=True, transform=None)

print(mnist_train.train_data.shape)
print(mnist_test.test_data.shape)

torch.Size([60000, 28, 28])
torch.Size([10000, 28, 28])


In [ ]:
class Tnet(nn.Module):
    def __init__(self, layer_num):
        super(Tnet, self).__init__()
        